In [116]:
import datetime as datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


plt.style.use('default') # Make the graphs a bit prettier
plt.rcParams['figure.figsize'] = (15, 5)

In [144]:
#condicionar algoritmos a usar
usarTodosLosDatos = True
modelo =0  #0: SGD_Regressor
             #1: Liner_Regressor

In [118]:
#cargamos datos tomando el ultimo archivo de cada año
if(not usarTodosLosDatos):
    dsSell2013 = pd.read_csv("sell2013.csv", low_memory=False)
    dsSell2014 = pd.read_csv("sell2014.csv", low_memory=False)
    dsSell2015 = pd.read_csv("sell2015.csv", low_memory=False)
    dsSell2016 = pd.read_csv("sell2016.csv", low_memory=False)
    dsSell2017 = pd.read_csv("sell2017.csv", low_memory=False)
    
    #unimos todos los csv y filtramos para que solo queda GBA y CABA
    dsSell = dsSell2013.append(dsSell2014).append(dsSell2015).append(dsSell2016).append(dsSell2017)

In [119]:
#cargamos todos los archivos de la pagina que se encuentren dentro de la carpeta enUso
import os
if(usarTodosLosDatos):
    dsSell = pd.DataFrame()
    for filename in os.listdir("datosCompletos/propieades/enUso"):
        dataSet = pd.read_csv("datosCompletos/propieades/enUso/" + filename, low_memory=False)
        dsSell = dsSell.append(dataSet)

In [120]:
dsSell.head()

,country_name,created_on,currency,description,expenses,extra,floor,geonames_id,id,image_thumbnail,...,price_per_m2,price_usd_per_m2,properati_url,property_type,rooms,state_name,surface_covered_in_m2,surface_in_m2,surface_total_in_m2,title
0,NaN,2012-08-08,NaN,NaN,850 aprox,NaN,10.0,3436077.0,NaN,http://cf-thumbs.properati.com.ar/Cry7pVKa41t_...,...,NaN,NaN,http://www.properati.com.ar/ag_venta_departame...,apartment,3.0,NaN,NaN,102.0,NaN,NaN
1,NaN,2012-08-08,NaN,NaN,NaN,NaN,11.0,3427467.0,NaN,http://cf-thumbs.properati.com.ar/MwrzW_dkQbqR...,...,NaN,NaN,http://www.properati.com.ar/ah_venta_departame...,apartment,3.0,NaN,NaN,73.0,NaN,NaN
2,NaN,2012-08-08,NaN,NaN,NaN,NaN,NaN,3427467.0,NaN,http://cf-thumbs.properati.com.ar/2wMqxAsh1N2t...,...,NaN,NaN,http://www.properati.com.ar/aj_venta_casa_vill...,house,4.0,NaN,NaN,485.0,NaN,NaN
3,NaN,2012-08-08,USD,NaN,NaN,NaN,NaN,3427467.0,NaN,http://cf-thumbs.properati.com.ar/uMwzJsvexa1C...,...,NaN,315.186246,http://www.properati.com.ar/ak_venta_casa_vill...,house,4.0,NaN,NaN,349.0,NaN,NaN
4,NaN,2012-08-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://cf-thumbs.properati.com.ar/4kPnofvgR6hs...,...,NaN,NaN,http://www.properati.com.ar/ao_venta_departame...,apartment,3.0,NaN,NaN,NaN,NaN,NaN


In [121]:
#Queremos quedarnos con la zona, el municipio y los que esten en CABA y GBA
def getMunicipioYZona(df):
    df['municipio'] = df['place_with_parent_names'].apply(lambda x: x.split('|')[3])
    df['zona'] = df['place_with_parent_names'].apply(lambda x: x.split('|')[2])

    df['filtro'] = df['zona'].apply(lambda x: ((x=='Capital Federal')or(x=='Bs.As. G.B.A. Zona Norte')or(x=='Bs.As. G.B.A. Zona Sur')or(x=='Bs.As. G.B.A. Zona Oeste')))
    return df[df.filtro]
    

In [122]:
#Como nos importa mucho el precio con respecto a otras cosas vamos a sacar a los que tienen esa columna en NaN
def filtrarPrecioNulo(df):
    df['tiene_precio'] = df['price'].apply(lambda x: pd.notnull(x))
    return df[df.tiene_precio]

In [123]:
#convertimos el formato de la fecha
def formatearFecha(df):
    df['year'] =pd.to_numeric(df['created_on'].map( lambda x: x.split('-')[0]))
    df['month'] = pd.to_numeric(df['created_on'].map( lambda x: x.split('-')[1]))
    df['created_on'] = df['created_on'] .map(lambda x: x.split('-')[0]+x.split('-')[1]+x.split('-')[2])


In [124]:
def getInfoFromDescription(df):
    df['cochera'] = df.description.str.contains('cochera')
    df['pileta'] = df.description.str.contains('pileta')
    df['sum'] = df.description.str.contains('sum')
    df['aire acondicionado'] = df.description.str.contains('aire acondicionado')
    df['calefaccion'] = df.description.str.contains('calefaccion')
    df['seguridad'] = df.description.str.contains('seguridad')
    df['estrenar'] = df.description.str.contains('estrenar')
    df['emprendimiento'] = df.description.str.contains('emprendimiento')

    #convertimos los valores false , true y nan de las busquedas anteriores

    df['cochera'] = df['cochera'].map(lambda x: 0 if x == 'False' else 1 if x == 'True' else 0.5)
    df['pileta'] = df['pileta'].map(lambda x: 0 if x == 'False' else 1 if x == 'True' else 0.5)
    df['sum'] = df['sum'].map(lambda x: 0 if x == 'False' else 1 if x == 'True' else 0.5)
    df['aire acondicionado'] = df['aire acondicionado'].map(lambda x: 0 if x == 'False' else 1 if x == 'True' else 0.5)
    df['calefaccion'] = df['calefaccion'].map(lambda x: 0 if x == 'False' else 1 if x == 'True' else 0.5)
    df['seguridad'] = df['seguridad'].map(lambda x: 0 if x == 'False' else 1 if x == 'True' else 0.5)
    df['estrenar'] = df['estrenar'].map(lambda x: 0 if x == 'False' else 1 if x == 'True' else 0.5)
    df['emprendimiento'] = df['emprendimiento'].map(lambda x: 0 if x == 'False' else 1 if x == 'True' else 0.5)
    

In [125]:
#pasar todas las superficies de surface_in_m2 a total_surface_in_m2
def estandarizarCampoSuperfice(df):
    mask = (df['surface_total_in_m2'].isnull())
    df['surface_total_in_m2'][mask] = df['surface_in_m2'][mask]

In [126]:
#drop de columnas
def dropColumnas(df):
    df.drop('price_per_m2', axis = 1, inplace = True)
    df.drop('place_with_parent_names', axis = 1, inplace = True)
    df.drop('created_on', axis = 1, inplace = True)
    df.drop('description', axis = 1, inplace = True)
    df.drop('currency', axis=1, inplace=True)
    df.drop('extra', axis=1, inplace=True)
    df.drop('id', axis=1, inplace=True)
    df.drop('image_thumbnail', axis=1, inplace=True)
    df.drop('properati_url', axis=1, inplace=True)
    df.drop('title', axis=1, inplace=True)
    df.drop('lat-lon', axis=1, inplace=True)
    df.drop('operation', axis=1, inplace=True)
    df.drop('price', axis=1, inplace=True)
    df.drop('price_aprox_local_currency', axis=1, inplace=True)
    df.drop('country_name', axis=1, inplace=True)
    df.drop('place_name', axis=1, inplace=True)
    df.drop('municipio', axis=1, inplace=True)
    df.drop('state_name', axis=1, inplace=True)
    df.drop('geonames_id', axis=1, inplace=True)
    df.drop('price_usd_per_m2', axis=1, inplace=True)
    df.drop('surface_in_m2', axis=1, inplace=True)
    df.drop('filtro', axis=1, inplace=True)
    df.drop('tiene_precio', axis=1, inplace=True)

In [127]:
dsSell.columns

Index(['country_name', 'created_on', 'currency', 'description', 'expenses',
       'extra', 'floor', 'geonames_id', 'id', 'image_thumbnail', 'lat',
       'lat-lon', 'lon', 'operation', 'place_name', 'place_with_parent_names',
       'price', 'price_aprox_local_currency', 'price_aprox_usd',
       'price_per_m2', 'price_usd_per_m2', 'properati_url', 'property_type',
       'rooms', 'state_name', 'surface_covered_in_m2', 'surface_in_m2',
       'surface_total_in_m2', 'title'],
      dtype='object')

In [128]:
#preprocessing

def preprocesamientoExpensas(df):
    #eliminamos todo lo q no es numero de expenses
    df['expenses'] = df['expenses'].str.extract('(\d+)')
    #convertimos a float

    df['expenses'] =pd.to_numeric(df.expenses, errors='coerce').notnull()
    

In [129]:
#one hot encoding de los place name. me suena mal muchas columnas. performance
def oneHotEncoding(df, columna):
  
    one_hot = pd.get_dummies(df[columna])
    df.drop(columna, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df
    

In [130]:
#distancia entre dos estaciones 
from math import radians, cos, sin, asin, sqrt

def distanciaLatLong(long1, lat1, long2, lat2):
    long1, lat1, long2, lat2 = map(radians, [long1, lat1, long2, lat2])
    dLong = long2 - long1 
    dLat = lat2 - lat1 
    a = sin(dLat/2)**2 + cos(lat1) * cos(lat2) * sin(dLong/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    return km 

In [131]:
def calcularEstaciones(reg):
    cantidad = 0
    for index, estacion in estacionesSubte.iterrows():
        cantidad = cantidad + 1 if (distanciaLatLong(reg[3], reg[2],estacion['X'], estacion['Y'])) < 0.4 else 0
    
    return cantidad
        

In [132]:
#cargamos los datos de las localizaciones de transporte y agregamos a dataset (por ahora solo cuenta la cantidad de estaciones)
#datos sacados de data.buenosaires.gob.ar
estacionesSubte = pd.read_csv("datosCompletos/transporte/estacionesSubte.csv", low_memory=False, encoding = 'ISO-8859-1')
estacionesTren = pd.read_csv("datosCompletos/transporte/estacionesTren.csv", low_memory=False,sep = ';')
def cargarLocalizacionesTransporte(df):
    df['cantidadSubtes'] = 0
    df['cantidadTrenes'] = 0
    cantidadEstacionesSubte = df.apply(lambda x:calcularEstaciones(x), axis = 1)
    df['cantidadSubtes'] = cantidadEstacionesSubte

In [133]:
#filleamos los nan con el promedio de las columnas
def fillNan(df):
    return df.apply(lambda x: x.fillna(x.mean()),axis=0)

In [134]:
dsSell = getMunicipioYZona(dsSell)
dsSell = filtrarPrecioNulo(dsSell)
formatearFecha(dsSell)
getInfoFromDescription(dsSell)
estandarizarCampoSuperfice(dsSell)
dropColumnas(dsSell)
preprocesamientoExpensas(dsSell)
dsSell = oneHotEncoding(dsSell, 'zona')
dsSell = oneHotEncoding(dsSell, 'property_type')
#cargarLocalizacionesTransporte(dsSell)
dsSell = fillNan(dsSell)

M:\Users\Krion\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
M:\Users\Krion\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


In [135]:
dsSell

,expenses,floor,lat,lon,price_aprox_usd,rooms,surface_covered_in_m2,surface_total_in_m2,year,month,...,estrenar,emprendimiento,Bs.As. G.B.A. Zona Norte,Bs.As. G.B.A. Zona Oeste,Bs.As. G.B.A. Zona Sur,Capital Federal,PH,apartment,house,store
3,False,17.634864,-34.536900,-58.529476,110000.00,4.000000,4310.434013,349.000000,2012,8,...,0.5,0.5,1,0,0,0,0,0,1,0
6,False,17.634864,-34.542278,-58.505344,99000.00,2.949446,4310.434013,153.000000,2012,8,...,0.5,0.5,1,0,0,0,0,0,1,0
10,False,17.634864,-34.536900,-58.529476,110000.00,2.949446,4310.434013,85.000000,2012,8,...,0.5,0.5,1,0,0,0,0,1,0,0
11,False,17.634864,-34.452498,-58.931356,650000.00,4.000000,4310.434013,383.000000,2012,8,...,0.5,0.5,1,0,0,0,0,0,1,0
12,False,17.634864,-34.451013,-58.626786,300000.00,2.949446,4310.434013,255.000000,2012,8,...,0.5,0.5,1,0,0,0,0,0,1,0
13,False,5.000000,-34.536900,-58.529476,600000.00,4.000000,4310.434013,240.000000,2012,8,...,0.5,0.5,0,0,0,1,0,1,0,0
14,False,17.634864,-34.626877,-58.478199,220000.00,5.000000,4310.434013,250.000000,2012,8,...,0.5,0.5,0,0,0,1,0,0,1,0
16,False,17.634864,-34.633633,-58.484419,51900.00,1.000000,4310.434013,25.000000,2012,8,...,0.5,0.5,0,0,0,1,0,1,0,0
17,False,17.634864,-34.608820,-58.430529,195000.00,4.000000,4310.434013,180.000000,2012,8,...,0.5,0.5,0,0,0,1,0,0,1,0
18,False,17.634864,-34.651912,-58.519874,115000.00,3.000000,4310.434013,108.000000,2012,8,...,0.5,0.5,0,0,0,1,0,1,0,0


In [136]:
#creamos un set de test y otro de entrenamiento. el test de entrenamiento sera de un 10% del original 

dsSell['is_true'] = np.random.uniform(0,1,len(dsSell)) >= 0.2
dsSellTrain, dsSellTest = dsSell[dsSell['is_true']==True], dsSell[dsSell['is_true']==False]
del dsSell['is_true']
del dsSellTrain['is_true']
del dsSellTest['is_true']

dsSellTrain_y = dsSellTrain['price_aprox_usd']
dsSellTrain_x = dsSellTrain.drop('price_aprox_usd', axis=1)
dsSellTest_y = dsSellTest['price_aprox_usd']
dsSellTest_x = dsSellTest.drop('price_aprox_usd', axis=1)

In [137]:
#normalize data
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model

dsSellTrain_x = preprocessing.normalize(dsSellTrain_x)
scaler = StandardScaler()
scaler.fit(dsSellTrain_x) 
dsSellTrain_x= scaler.transform(dsSellTrain_x)


In [149]:
#usar SGD_Regressor
if (modelo == 0):
    params = {'loss' : ['squared_epsilon_insensitive'],
              'learning_rate' : ['invscaling', 'optimal'] , 'eta0' : [0.01,0.001,0.1,1,10,5], 'power_t' : [0.25,0.3,0.8,0.1,0.1,0.60],
                'alpha' : [1,10,0.01,0.0001,1000,100,200]} 
    model = linear_model.SGDRegressor()

In [150]:
#usar Linear_Regressor
if (modelo == 1):
    params = {} 
    model = linear_model.LinearRegression(normalize = True)

In [152]:
#random search del modelo elegido
from sklearn.model_selection import RandomizedSearchCV
from numpy import random

from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer

import scipy
import time

print("[INFO] tuning hyperparameters via grid search")


grid = RandomizedSearchCV(model, params, n_jobs = -1, n_iter = 50)


start = time.time()
print("[INFO] fitting stage")
grid = grid.fit(dsSellTrain_x , dsSellTrain_y)
acc = grid.best_score_
model.get_params()
print("[INFO] grid search took {:.2f} seconds".format(time.time() - start))
print("[INFO] grid search accuracy: {:.2f}%".format(acc * 100))
print("[INFO] grid search best parameters: {}".format(grid.best_params_))

[INFO] tuning hyperparameters via grid search
[INFO] fitting stage
[INFO] grid search took 109.27 seconds
[INFO] grid search accuracy: 21.00%
[INFO] grid search best parameters: {'power_t': 0.6, 'loss': 'squared_epsilon_insensitive', 'learning_rate': 'invscaling', 'eta0': 0.1, 'alpha': 0.01}


In [99]:
#levantamos el set de test 
testSet = pd.read_csv("testSet.csv", low_memory=False)
testId = testSet['id']

In [89]:
#drop de columnas
def dropColumnasTestSet(df):
    df.drop('place_with_parent_names', axis = 1, inplace = True)
    df.drop('created_on', axis = 1, inplace = True)
    df.drop('description', axis = 1, inplace = True)
    df.drop('id', axis=1, inplace=True)
    df.drop('lat-lon', axis=1, inplace=True)
    df.drop('operation', axis=1, inplace=True)
    df.drop('country_name', axis=1, inplace=True)
    df.drop('place_name', axis=1, inplace=True)
    df.drop('municipio', axis=1, inplace=True)
    df.drop('state_name', axis=1, inplace=True)
    df.drop('filtro', axis=1, inplace=True)

In [90]:
#preprocesamos el set de test
testSetPreprocessed = testSet

getMunicipioYZona(testSetPreprocessed)
formatearFecha(testSetPreprocessed)
getInfoFromDescription(testSetPreprocessed)
dropColumnasTestSet(testSetPreprocessed)
preprocesamientoExpensas(testSetPreprocessed)
oneHotEncoding(testSetPreprocessed, 'zona')
oneHotEncoding(testSetPreprocessed, 'property_type')
testSetPreprocessed = fillNan(testSetPreprocessed)

M:\Users\Krion\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


In [91]:
testSetPreprocessed.columns

Index(['lat', 'lon', 'surface_total_in_m2', 'surface_covered_in_m2', 'floor',
       'rooms', 'expenses', 'year', 'month', 'cochera', 'pileta', 'sum',
       'aire acondicionado', 'calefaccion', 'seguridad', 'estrenar',
       'emprendimiento'],
      dtype='object')

In [92]:
testPrice= grid.predict(testSetPreprocessed)

In [93]:
dataPredictions = {'id' : testId,
        'price_usd' : testPrice}
testPrediction = pd.DataFrame(dataPredictions)

In [94]:
testPrediction

,id,price_usd
0,3632,1495624.506
1,3633,1495594.747
2,2263404,2083519.427
3,2263405,1903678.591
4,2263406,2006382.566
5,2263407,1903678.591
6,2263408,1903678.591
7,2263409,1903678.591
8,2263410,2202299.361
9,2263411,1972147.907


In [ ]:
#exportar csv con formato para kaggle
fileName = "predictionSGDRegressor.csv"
testPrediction.to_csv(fileName, sep=',')